In [22]:
!pip install -q transformers accelerate bitsandbytes peft torchinfo torchmetrics
!pip install -q pytorch-lightning wandb

In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional, Dict
import math
from collections import defaultdict
import time

# PyTorch ecosystem imports
from torch.cuda.amp import GradScaler  # Mixed precision training
from torch import amp
from transformers import AutoTokenizer, GPT2Tokenizer  # HuggingFace tokenizers
import torch._dynamo as dynamo  # For torch.compile

# Set seeds
np.random.seed(42)
torch.manual_seed(42)

# Colab GPU detection and setup
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(f"GPU: {torch.cuda.get_device_name(0)}")
  print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
  device = torch.device('cpu')
  print("No GPU available, using CPU")

GPU: Tesla T4
Memory: 14.7 GB


# Autoregressive Generation

In [24]:
class SimpleLanguageModel:
  """Vanilla Autoregressive text generation"""

  def __init__(self, vocab_size: int, context_length:int ):
    self.vocab_size = vocab_size
    self.context_length = context_length

    # Simple bigram probs as transition matrix. A “transition matrix” originates from Markov chain theory: A Markov chain is a stochastic process where the probability of the next state depends only on the current state (the Markov property). In a Markov chain with 𝑛 n states, the transition matrix encodes all the probabilities of moving from one state to another. Entry 𝑃 𝑖 𝑗 P ij ​ = probability of transitioning from state 𝑖 i to state 𝑗 j
    self.transition_matrix = np.random.dirichlet(np.ones(vocab_size), size=vocab_size) # [V, V] stochastic matrix
    print(f"Transition matrix shape: {self.transition_matrix.shape}")
    print(f"Sample row (sums to 1): {self.transition_matrix[0,:]}  sum={self.transition_matrix[0].sum():.3f}")

  def get_next_token_probs(self, context: List[int]) -> np.ndarray:
    """Get prob distrib for next token given the context"""

    if len(context) == 0:
      ## If no context apply Uniform distrib
      return(np.ones(self.vocab_size)/self.vocab_size)

    last_token = context[-1] # Bigram uses just the latest token to predict the next
    print(f"Last token: {last_token}, vocab_size: {self.vocab_size}")

    probs = self.transition_matrix[last_token] # prob P(next|last)
    print(f"\nContext: {context} -> Last token: {last_token}")
    print(f"Next token probs: {probs[:5]} ... (showing first 5)")
    return probs

  def generate(self, prompt: List[int], max_new_tokens: int, strategy: str = 'greedy') -> List[int]:
    """Generate tokens autoregressively"""

    context = prompt.copy()

    for i in range(max_new_tokens):
      # get prob distrib for nxt token
      probs = self.get_next_token_probs(context[-self.context_length:]) # slicing and using the last context_length tokens in the prompt(=context)


      if   strategy == 'greedy':
        next_token = np.argmax(probs) # Choose most likely, deterministic
        print(f"Step {i}: Greedy selected token {next_token} with p={probs[next_token]:.3f}")

      elif strategy == 'sample':
        next_token = np.random.choice(self.vocab_size, p=probs) # Sample from ditrib
        print(f"Step {i}: Sampled token {next_token} with p={probs[next_token]:.3f}")


      context.append(next_token) # Starts from the prompt/context and keeps generating increasing it

      return(context)

##Simple Test

In [25]:
# Test vanilla implementation
vocab_size = 10  # Small vocab for visualization
model = SimpleLanguageModel(vocab_size=vocab_size, context_length=5)

prompt = [1, 2, 3]
print(f"\nGenerating from prompt: {prompt}\n")

# Compare greedy vs sampling
greedy_output = model.generate(prompt, max_new_tokens=5, strategy='greedy')
print(f"\nGreedy output: {greedy_output}")

sampled_output = model.generate(prompt, max_new_tokens=5, strategy='sample')
print(f"\nSampled output: {sampled_output}")

Transition matrix shape: (10, 10)
Sample row (sums to 1): [0.04569443 0.29310702 0.12821656 0.0888967  0.01651702 0.01651424
 0.00582673 0.19584123 0.08949454 0.11989152]  sum=1.000

Generating from prompt: [1, 2, 3]

Last token: 3, vocab_size: 10

Context: [1, 2, 3] -> Last token: 3
Next token probs: [0.08215403 0.01642155 0.00590806 0.26119085 0.2960559 ] ... (showing first 5)
Step 0: Greedy selected token 4 with p=0.296

Greedy output: [1, 2, 3, np.int64(4)]
Last token: 3, vocab_size: 10

Context: [1, 2, 3] -> Last token: 3
Next token probs: [0.08215403 0.01642155 0.00590806 0.26119085 0.2960559 ] ... (showing first 5)
Step 0: Sampled token 0 with p=0.082

Sampled output: [1, 2, 3, 0]


# Autoregressive Generation with torch.compile and HuggingFace tokenizers

In [26]:
from torch.cuda.amp import GradScaler # mix precision
from torch import amp
from transformers import AutoTokenizer, GPT2Tokenizer

class OptimizedLanguageModel(nn.Module):

  def __init__(self, vocab_size:int, d_model:int, context_length:int):
    super(OptimizedLanguageModel, self).__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.context_length = context_length

    # Model components
    self.token_embedding = nn.Embedding(vocab_size, d_model)
    self.positional_embedding = nn.Embedding(context_length, d_model)
    self.attention = nn.MultiheadAttention(d_model, num_heads=4, batch_first=True)
    self.ln = nn.LayerNorm(d_model)
    self.lm_head = nn.Linear(d_model, vocab_size)

    print(f"Model params: {sum(p.numel() for p in self.parameters()):,}")

  def forward(self, idx:torch.Tensor) -> torch.Tensor:
    B, T = idx.shape

    tok_emb = self.token_embedding(idx) # [B,T,d_model]

    pos = torch.arange(T, device=idx.device)
    pos_emb = self.positional_embedding(pos)

    x = tok_emb + pos_emb

    # Causal mask
    mask = torch.triu(torch.ones(T,T,device=idx.device),diagonal=1).bool()
    x, _ = self.attention(x,x,x, attn_mask=mask) # scaled dot-product attention mechanism: [Q, K,V]

    logits = self.lm_head(x) # [B, T, vocab_size]

    return(logits)



## Using HF pretrained tokenizers
- Rust backend = Fast
- Manages special tokens, padding and truncation automatically.
- Good for edge cases, multilingual, fast.

In [27]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

print(f"\nTokenizer details:")
print(f"  Vocab size: {tokenizer.vocab_size}")
print(f"  Model max length: {tokenizer.model_max_length}")
print(f"  Special tokens: {tokenizer.special_tokens_map}")
print(f"  Example encoding: 'Hello world!' -> {tokenizer.encode('Hello world!')}")
print(f"  Decoded back: {tokenizer.decode(tokenizer.encode('Hello world!'))}")


# create and compile model
model = OptimizedLanguageModel(
    vocab_size=tokenizer.vocab_size,
    d_model=256,
    context_length=128
).to(device)


# torch.compile options:
# - default: balanced optim
# - reduce-overhead: best for small models, minimizes kernel launch overhead
# - max_autotune: best for large models, spends more time finding optimial kernels
# - max-autotune-no-cudagraphs: flexible, max optimization without cuda graphs

if hasattr(torch, 'compile') and device.type == 'cuda':
  # Compile only makes sense on GPU (uses Triton compiler for CUDA)
  model_compiled = torch.compile(model, mode="reduce-overhead")
  print("\n Model compiled with torch.compile (reduce-overhead mode)")
  print("  First call will be slow (compilation), subsequent calls fast")
else:
  model_compiled = model
  print("\ntorch.compile not available or on CPU, using eager mode")


# Autom Mixed Precision (amp)
# master weights in FLP32
# forward in FLP16
# Automatic casting based on operation
# loss scaling prevents grad underflow
# good for Tensor Cores, with too large models not fitting in mem and in Training

def generate_with_amp(
    model: nn.Module,
    prompt: str,
    tokenizer,
    max_new_tokens: int=50,
    temperature: float = 1.0) -> str:

  """Generate text with automatic mixed precision (AMP) """

  input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device) # pt for PyTorch, there is also for np and tf

  with torch.no_grad():
    # autocast
    with amp.autocast(device_type=device.type, enabled=(device.type=='cuda')):
      for i in range(max_new_tokens):
        # runs in float16 but some ops like softmax stay in float32
        logits = model_compiled(input_ids)
        logits = logits[:, -1, :] / temperature # Scaling with temp

        probs = F.softmax(logits, dim=-1) # softmax in float32  for stability
        next_token = torch.multinomial(probs, num_samples = 1)

        input_ids = torch.cat([input_ids, next_token], dim=1)

        if next_token.item() == tokenizer.eos_token_id:
          break

  return tokenizer.decode(input_ids[0], skip_special_tokens=True)



Tokenizer details:
  Vocab size: 50257
  Model max length: 1024
  Special tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}
  Example encoding: 'Hello world!' -> [15496, 995, 0]
  Decoded back: Hello world!
Model params: 26,078,289

 Model compiled with torch.compile (reduce-overhead mode)
  First call will be slow (compilation), subsequent calls fast


## Comparing compiled vs eager mode

In [28]:
def benchmark_inference(model, compiled_model, input_ids, n_runs=10):

  # Warmup - triggers compilation
  print("Warming up (triggering compilation)...")
  for _ in range(2): # When benchmarking models, the first few runs are usually slower due to: Lazy initialization (loading kernels, allocating memory). JIT compilation overhead (torch.compile, TensorRT, XLA, etc). Running a couple of warmup iterations ensures those costs are paid upfront, so later timings reflect steady-state performance
    _ = model(input_ids)
    _ = compiled_model(input_ids)


  # Eager mode timing
  if device.type == 'cuda':
    torch.cuda.synchronize() # waiting for gpu to finish, cuda runs async

  start = time.time()
  for _ in range(n_runs): # Runs the model n_runs times on the same input_ids. Used to measure average runtime or variance across runs
    with torch.no_grad(): # not training
      _ = model(input_ids)

  if device.type == 'cuda':
    torch.cuda.synchronize() # waiting for gpu to finish, cuda runs async

  eager_time = time.time() -start


  # Compiled mode timing
  if device.type == 'cuda':
    torch.cuda.synchronize() # waiting for gpu to finish, cuda runs async

  start = time.time()
  for _ in range(n_runs): # Runs the model n_runs times on the same input_ids. Used to measure average runtime or variance across runs
    with torch.no_grad(): # not training
      _ = compiled_model(input_ids)

  if device.type == 'cuda':
    torch.cuda.synchronize()
  compiled_time = time.time() - start

  print(f"\nInference benchmark ({n_runs} runs):")
  print(f"  Eager mode: {eager_time:.3f}s ({1000*eager_time/n_runs:.1f}ms per call)")
  print(f"  Compiled: {compiled_time:.3f}s ({1000*compiled_time/n_runs:.1f}ms per call)")
  if compiled_time > 0:
    print(f"  Speedup: {eager_time/compiled_time:.2f}x")




### Test on realistic input

In [38]:
test_input = torch.randint(0, tokenizer.vocab_size, (1, 32)).to(device)
benchmark_inference(model, model_compiled, test_input)

# generate
prompt = "The future of AI is"
generated = generate_with_amp(model_compiled, prompt, tokenizer, max_new_tokens=20)
print(f"\nGenerated: {generated}")

print("\nNote: Have to do the benchmark with longer sequences or multiple prompts to see real gains.")

Warming up (triggering compilation)...

Inference benchmark (10 runs):
  Eager mode: 0.008s (0.8ms per call)
  Compiled: 0.007s (0.7ms per call)
  Speedup: 1.10x

Generated: The future of AI is suspect Education Ib sympathetic finRumPassword kins viewers phantomraseDead microbiotacknow FedExا� pun baconalk leaders

Note: Have to do the benchmark with longer sequences or multiple prompts to see real gains.
